![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# **DocumentMLClassifierApproach**

This notebook will cover the different parameters and usages of `DocumentMLClassifierApproach`.

**📖 Learning Objectives:**

1. Understand how to train a model to classify documents with a Logarithmic Regression algorithm.

2. Become comfortable using the different parameters of the annotator.


**🔗 Helpful Links:**

- Python Docs : [DocumentMLClassifierApproach](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/classification/document_ml_classifier/index.html)

- Scala Docs : [DocumentMLClassifierApproach](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/classification/DocumentMLClassifierApproach.html)

- For extended examples of usage, see the [Spark NLP Workshop repository](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/healthcare-nlp).

## **🎬 Colab Setup**

In [1]:
!pip install -q johnsnowlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.5 

In [2]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving spark_nlp_for_healthcare_spark_ocr_7139.json to spark_nlp_for_healthcare_spark_ocr_7139.json


In [3]:
from johnsnowlabs import nlp

nlp.install()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_7139.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_7139.json
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-5.1.4-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==5.1.4 installed! ✅ Heal the planet with NLP! 


In [1]:
from johnsnowlabs import nlp, medical
import pyspark.sql.functions as F
import pandas as pd

spark = nlp.start()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_7139.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.1.4, 💊Spark-Healthcare==5.1.4, running on ⚡ PySpark==3.1.2


## **🖨️ Input/Output Annotation Types**

- Input: `TOKEN`

- Output: `CATEGORY`

## **🔎 Parameters**


- `labelCol`: (str) Sets column with the value result we are trying to predict.

- `maxIter`: (Int) Sets maximum number of iterations.

- `tol`: (float) Sets convergence tolerance after each iteration.

- `fitIntercept`: (str) Sets whether to fit an intercept term, default is true.

- `classificationModelClass`: (str) Sets a the classification model class from SparkML to use; possible values are: `logreg` (Logistic Regression) or `svm` (Support Vector Machines). Defaults to `svm`.

- `minTokenNgram`: (int) Sets minimum number of tokens for Ngrams.

- `maxTokenNgram`: (int) Sets maximum number of tokens for Ngrams.

- `mergeChunks`: (boolean) whether to merge all chunks in a document or not (Default: false)

## Prepare Data

We will use a dataset with Adverse Drug Events (ADE) examples to train a binary classification model (contains ADE or not).

In [2]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

In [3]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


In [4]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


In [6]:
ade_df= pd.concat([df_neg, df_pos]).sample(frac=1) #merge and shuffle the data
ade_df.head()

,text,category
12142,"We conclude that, though rare, E. avium can b...",neg
13697,We report on a case of an adult patient treat...,neg
4709,The risk of cardiac decompensation in infants ...,pos
2654,This is the first reported case of BOOP associ...,pos
11773,Implications for its drug management are disc...,neg


In [7]:
ade_df.category.value_counts()

neg    16695
pos     6821
Name: category, dtype: int64

Send the data to a spark data frame:

In [8]:
spark_df = spark.createDataFrame(ade_df)
spark_df.show(3)

+--------------------+--------+
|                text|category|
+--------------------+--------+
| We conclude that...|     neg|
| We report on a c...|     neg|
|The risk of cardi...|     pos|
+--------------------+--------+
only showing top 3 rows



In [9]:
train_data, test_data = spark_df.randomSplit([0.7, 0.3], seed=100)

print("Train Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Train Dataset Count: 16464
Test Dataset Count: 7052


Now, let's check how to train models using different hyperparameters.

### `setlabelCol()`



We could set text preprocessing stages in the pipeline (stopword removal, stemming, lemmatization, etc.), but for simplicity we keep the pipeline with tokenization only.

The first parameter to understand is how to set the correct column of the data frame that contains the ground truth label of the text.

In [10]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

token = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

classifier = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction")

pipeline = nlp.Pipeline(stages=[document, token, classifier])

Train the model:

In [11]:
%%time
model = pipeline.fit(train_data)


CPU times: user 362 ms, sys: 55.8 ms, total: 418 ms
Wall time: 58.5 s


In [12]:
result = model.transform(test_data).cache()
result.show()

+--------------------+--------+--------------------+--------------------+--------------------+
|                text|category|            document|               token|          prediction|
+--------------------+--------+--------------------+--------------------+--------------------+
| A 50-year-old wo...|     neg|[{document, 0, 11...|[{token, 1, 1, A,...|[{category, 1, 11...|
| A 56-yr-old whit...|     neg|[{document, 0, 19...|[{token, 1, 1, A,...|[{category, 1, 19...|
| Chloroquine trea...|     neg|[{document, 0, 96...|[{token, 1, 11, C...|[{category, 1, 96...|
| In particular, c...|     neg|[{document, 0, 12...|[{token, 1, 2, In...|[{category, 1, 12...|
| The association ...|     neg|[{document, 0, 12...|[{token, 1, 3, Th...|[{category, 1, 12...|
| The catheter had...|     neg|[{document, 0, 16...|[{token, 1, 3, Th...|[{category, 1, 16...|
| The median age w...|     neg|[{document, 0, 42...|[{token, 1, 3, Th...|[{category, 1, 42...|
| The tube was rem...|     neg|[{document, 0, 17..

Evaluating the model

In [13]:
from sklearn.metrics import classification_report

In [14]:
result_df = result.select('category','prediction.result').toPandas()
result_df['result'] = result_df.result.apply(lambda x : x[0])

print (classification_report(result_df['category'], result_df['result']))

              precision    recall  f1-score   support

         neg       0.87      0.97      0.92      5013
         pos       0.90      0.65      0.76      2039

    accuracy                           0.88      7052
   macro avg       0.89      0.81      0.84      7052
weighted avg       0.88      0.88      0.87      7052



### `setClassificationModelClass()`



By default, the annotator wil train a `SVM` model, but we can also set it to `logreg`.

In [15]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

token = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

In [16]:
%%time
logreg_model = pipeline.fit(train_data)

CPU times: user 214 ms, sys: 25.4 ms, total: 239 ms
Wall time: 31.1 s


In [17]:
result_logreg = logreg_model.transform(test_data).cache()
result_logreg.show()

+--------------------+--------+--------------------+--------------------+--------------------+
|                text|category|            document|               token|          prediction|
+--------------------+--------+--------------------+--------------------+--------------------+
| A 50-year-old wo...|     neg|[{document, 0, 11...|[{token, 1, 1, A,...|[{category, 1, 11...|
| A 56-yr-old whit...|     neg|[{document, 0, 19...|[{token, 1, 1, A,...|[{category, 1, 19...|
| Chloroquine trea...|     neg|[{document, 0, 96...|[{token, 1, 11, C...|[{category, 1, 96...|
| In particular, c...|     neg|[{document, 0, 12...|[{token, 1, 2, In...|[{category, 1, 12...|
| The association ...|     neg|[{document, 0, 12...|[{token, 1, 3, Th...|[{category, 1, 12...|
| The catheter had...|     neg|[{document, 0, 16...|[{token, 1, 3, Th...|[{category, 1, 16...|
| The median age w...|     neg|[{document, 0, 42...|[{token, 1, 3, Th...|[{category, 1, 42...|
| The tube was rem...|     neg|[{document, 0, 17..

In [18]:
result_df = result_logreg.select('category','prediction.result').toPandas()
result_df['result'] = result_df.result.apply(lambda x : x[0])

print (classification_report(result_df['category'], result_df['result']))

              precision    recall  f1-score   support

         neg       0.86      0.98      0.92      5013
         pos       0.93      0.62      0.75      2039

    accuracy                           0.88      7052
   macro avg       0.90      0.80      0.83      7052
weighted avg       0.88      0.88      0.87      7052



### `setMaxIter()`

maximum number of iterations (Default: 10), affects the training time and convergence.

In [19]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

token = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

classifier_svm = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")\
    .setMaxIter(5)

pipeline = nlp.Pipeline(stages=[document, token, classifier_svm])

result_svm = pipeline.fit(train_data).transform(test_data)

result_svm_df = result_svm.select('category','prediction.result').toPandas()
result_svm_df['result'] = result_svm_df.result.apply(lambda x : x[0])

print (classification_report(result_svm_df['category'], result_svm_df['result']))

              precision    recall  f1-score   support

         neg       0.86      0.98      0.92      5013
         pos       0.93      0.62      0.74      2039

    accuracy                           0.88      7052
   macro avg       0.90      0.80      0.83      7052
weighted avg       0.88      0.88      0.87      7052



### `setTol()`

convergence tolerance after each iteration (Default: 1e-6)

In [20]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

token = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setTol(1e-3)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.86      0.98      0.92      5013
         pos       0.93      0.62      0.75      2039

    accuracy                           0.88      7052
   macro avg       0.90      0.80      0.83      7052
weighted avg       0.88      0.88      0.87      7052



### `setMinTokenNgram()` and `setMaxTokenNgram()`

In [21]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

token = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMinTokenNgram(1)\
    .setMaxTokenNgram(3)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_pf = result_logreg.select('category','prediction.result').toPandas()
result_logreg_pf['result'] = result_logreg_pf.result.apply(lambda x : x[0])

print (classification_report(result_logreg_pf['category'], result_logreg_pf['result']))

              precision    recall  f1-score   support

         neg       0.86      0.98      0.92      5013
         pos       0.93      0.62      0.75      2039

    accuracy                           0.88      7052
   macro avg       0.90      0.80      0.83      7052
weighted avg       0.88      0.88      0.87      7052



### `setMergeChunks()`

whether to merge all chunks in a document or not (Default: false)

In [22]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

token = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMergeChunks(True)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_pf = result_logreg.select('category','prediction.result').toPandas()
result_logreg_pf['result'] = result_logreg_pf.result.apply(lambda x : x[0])

print (classification_report(result_logreg_pf['category'], result_logreg_pf['result']))

              precision    recall  f1-score   support

         neg       0.86      0.98      0.92      5013
         pos       0.93      0.62      0.75      2039

    accuracy                           0.88      7052
   macro avg       0.90      0.80      0.83      7052
weighted avg       0.88      0.88      0.87      7052



### `setFitIntercept()`



whether to fit an intercept term (Default: true)

In [23]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

token = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setFitIntercept(False)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_pf = result_logreg.select('category','prediction.result').toPandas()
result_logreg_pf['result'] = result_logreg_pf.result.apply(lambda x : x[0])

print (classification_report(result_logreg_pf['category'], result_logreg_pf['result']))

              precision    recall  f1-score   support

         neg       0.87      0.97      0.92      5013
         pos       0.91      0.64      0.75      2039

    accuracy                           0.88      7052
   macro avg       0.89      0.81      0.84      7052
weighted avg       0.88      0.88      0.87      7052

